##The Entire Object Detection Pipeline in one Notebook

The goal of this notebook is to streamline the pipeline from labeled .xml files to training and testing a model. 

Some important aspects of this notebook include:

* For now, we use **TensorFlow 1.15.0** throughout the pipeline
* To minimize changes to this notebook, follow the folder structure:
* We need to modify our PYTHONPATH to use the the ***models*** package

In [0]:
#mount google drive with folder structure
from google.colab import drive
drive.mount('/content/drive')
%cd 'drive/My Drive/NanocrystalDetection/PdO_detection'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[Errno 2] No such file or directory: 'drive/My Drive/NanocrystalDetection/PdO_detection'
/content/drive/My Drive/NanocrystalDetection/Old_Method/models/research/object_detection


In [0]:
#install and confirm the version of tensorflow
!pip3 install tensorflow==1.15
import tensorflow as tf
assert tf.__version__ == '1.15.0',"Not the right version of tensorflow!"

1.15.0


In [0]:
#also need to fix $PATH, $PYTHONPATH
#PATH: /usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin:/opt/bin
#PYTHONPATH: /env/python/content/drive/My Drive/NanocrystalDetection/Old_Method/models/research/content/drive/My Drive/NanocrystalDetection/Old_Method/models/research/slim
import os
!protoc object_detection/protos/*.proto --python_out=.
os.environ['PYTHONPATH'] += "/content/drive/My Drive/NanocrystalDetection/Old_Method/models/research"
os.environ['PYTHONPATH'] += "/content/drive/My Drive/NanocrystalDetection/Old_Method/models/research/slim"
#!echo $PATH
#!echo $PYTHONPATH

/usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin:/opt/bin
/env/python
object_detection/protos/*.proto: No such file or directory
/usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin:/opt/bin
/env/python/content/drive/My Drive/NanocrystalDetection/Old_Method/models/research/content/drive/My Drive/NanocrystalDetection/Old_Method/models/research/slim


In [0]:
#installing research directory takes 2-3 minutes
%cd /content/drive/My Drive/NanocrystalDetection/Old_Method/models/research
!python3 setup.py install

In [0]:
#installing slim takes < 1 min
%cd /content/drive/My Drive/NanocrystalDetection/Old_Method/models/research/slim
!python3 setup.py install

In [0]:
#this checks that your environment is set up OK!
!pwd #(/content/drive/My Drive/NanocrystalDetection/Old_Method/models/research)
%cd /content/drive/My Drive/NanocrystalDetection/Old_Method/models/research
!python3 object_detection/builders/model_builder_test.py

In [0]:
#now we finally train our model using GPU compute!
#need to be in the training_demo directory
%cd '/content/drive/My Drive/NanocrystalDetection/Old_Method/workingspace_PdO_forColab/training_demo'
!python3 train.py --logtostderr --train_dir='/content/drive/My Drive/NanocrystalDetection/Old_Method/workingspace_PdO_forColab/training_demo/training/' --pipeline_config_path='/content/drive/My Drive/NanocrystalDetection/Old_Method/workingspace_PdO_forColab/training_demo/training/ssd_inception_v2_coco.config'

In [0]:
#make sure to change checkpoint number if necessary!
%cd '/content/drive/My Drive/NanocrystalDetection/Old_Method/workingspace_PdO_forColab/training_demo'
!python3 export_inference_graph.py --input_type image_tensor --pipeline_config_path training/ssd_inception_v2_coco.config --trained_checkpoint_prefix training/model.ckpt-19283 --output_directory trained-inference-graphs/output_inference_graph_v1.pb

In [0]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior() #doing this fixes certain gfile issues
import zipfile
import tensorflow.io as tf_io
from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

Instructions for updating:
non-resource variables are not supported in the long term


In [0]:
# This is needed to display the images.
%matplotlib inline

# This is needed since the notebook is stored in the object_detection folder.
#sys.path.append("..")

In [0]:
%cd /content/drive/My Drive/NanocrystalDetection/Old_Method/models/research/object_detection
from utils import label_map_util
from utils import visualization_utils as vis_util

/content/drive/My Drive/NanocrystalDetection/Old_Method/models/research/object_detection


In [0]:
# What model to download.
MODEL_NAME = 'ssd_mobilenet_v2_coco_2018_03_29'
MODEL_FILE = MODEL_NAME + '.tar.gz'

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_CKPT = '/content/drive/My Drive/NanocrystalDetection/Old_Method/workingspace_PdO_forColab/training_demo/trained-inference-graphs/output_inference_graph_v1.pb/frozen_inference_graph.pb' #this is the second model I trained, and was successful!

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = '/content/drive/My Drive/NanocrystalDetection/Old_Method/workingspace_PdO_forColab/training_demo/annotations/label_map.pbtxt' #this label maps, for now, only has raccoons in it!

NUM_CLASSES = 1

In [0]:
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

In [0]:
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

In [0]:
def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

In [0]:
PATH_TO_TEST_IMAGES_DIR = '/content/drive/My Drive/NanocrystalDetection/Old_Method/workingspace_PdO_forColab/training_demo/untouched3_PdO_dense_newmaterial'
TEST_IMAGE_PATHS = [ os.path.join(PATH_TO_TEST_IMAGES_DIR, '{}.jpg'.format(i)) for i in range(1, 88) ]

lengths = []
aspect_ratios = []

with detection_graph.as_default():
    with tf.Session(graph=detection_graph) as sess:
        for image_path in TEST_IMAGE_PATHS:
            #print(image_path)
            image = Image.open(image_path)
            # the array based representation of the image will be used later in order to prepare the
            # result image with boxes and labels on it.
            image_np = load_image_into_numpy_array(image)
            # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
            image_np_expanded = np.expand_dims(image_np, axis=0)
            image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
            # Each box represents a part of the image where a particular object was detected.
            boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
            # Each score represent how level of confidence for each of the objects.
            # Score is shown on the result image, together with the class label.
            scores = detection_graph.get_tensor_by_name('detection_scores:0')
            classes = detection_graph.get_tensor_by_name('detection_classes:0')
            num_detections = detection_graph.get_tensor_by_name('num_detections:0')
            # Actual detection.
            (boxes, scores, classes, num_detections) = sess.run(
                [boxes, scores, classes, num_detections],
                feed_dict={image_tensor: image_np_expanded})
            # Visualization of the results of a detection.
            vis_util.visualize_boxes_and_labels_on_image_array(
                image_np,
                np.squeeze(boxes),
                np.squeeze(classes).astype(np.int32),
                np.squeeze(scores),
                category_index,
                use_normalized_coordinates=True,
                line_thickness=1,
                )
            plt.figure(figsize=(12, 9))
            plt.imshow(image_np)
            for i in range(scores.shape[1]):
                if scores[0, i]*100 > 50:
                    figure_edge_length = 139##CRITICAL TO CHANGE
                    d1 = round(figure_edge_length*np.abs(boxes[0, i, 0] - boxes[0, i, 2]), 1)
                    d2 = round(figure_edge_length*np.abs(boxes[0, i, 1] - boxes[0, i, 3]), 1)
                    #print("Nanoparticle dimensions are {} nm by {} nm".format(d1, d2))
                    lengths.append(d1)
                    aspect_ratio = np.max([d1/d2, d2/d1])
                    aspect_ratios.append(aspect_ratio)
                    #lengths.append(d2)
                    #print("Appending {} nm".format((d1+d2)/2))
            print("Now mean {} and standard deviation {}".format(np.mean(lengths), np.std(lengths)))
plt.figure(figsize=(9,4.5))
plt.subplot(1, 2, 1)  # 1 line, 2 rows, index nr 1 (first position in the subplot)
plt.hist(lengths, 60)
plt.xlabel("Diameter (nm)")
plt.ylabel("Counts")
plt.subplot(1, 2, 2)  # 1 line, 2 rows, index nr 2 (second position in the subplot)
plt.hist(aspect_ratios, 60)
plt.xlabel("Aspect Ratio")
plt.ylabel("Counts")
plt.show()

print('Final mean NC size is {} nm and standard deviation is {} percent given {} measurements taken!'.format(np.mean(lengths), np.std(lengths)/np.mean(lengths)*100, len(lengths)))